In [13]:
import numpy as np
import pandas as pd
import random
import time
from IPython.display import clear_output

from gym import Env, spaces
from gym.utils import seeding
from gym.envs.toy_text import discrete

from Fairness_Metrics import fairness_metric
import warnings
warnings.filterwarnings('ignore')

In [14]:
data = pd.read_csv('csv/IA_counties.csv')
data.head()

,X,Y,STATEFP10,COUNTYFP10,GEOID10,NAME10,NAMELSAD10,ALAND10,AWATER10,INTPTLAT10,...,PRES08OTH,TOTVOT12,PRES12D,PRES12R,PRES12OTH,TOTVOT16,PRES16D,PRES16R,PRES16OTH,CD
0,-92.999001,42.035801,19,127,19127,Marshall,Marshall County,1482770678,1803086,42.041691,...,356,19064,10257,8472,335,17980,7652,9146,1182,1
1,-92.066966,42.080299,19,11,19011,Benton,Benton County,1855117342,5760770,42.092547,...,207,14023,6862,6940,221,13844,4678,8232,934,1
2,-95.150846,43.082520,19,41,19041,Clay,Clay County,1469139214,13866941,43.079822,...,118,8502,3385,4951,166,8617,2249,5877,491,4
3,-95.325464,41.684636,19,165,19165,Shelby,Shelby County,1530110414,1486135,41.679014,...,102,6483,2469,3911,103,6370,1662,4362,346,4
4,-91.352586,42.862146,19,43,19043,Clayton,Clayton County,2016405612,36586071,42.840998,...,143,9138,4806,4164,168,9129,3237,5317,575,1


In [15]:
# Creating average votes for both the parties 

req_data = pd.DataFrame({})
req_data['d_votes'] = data['PRES16D'] / data['TOTVOT16']
req_data['r_votes'] = data['PRES16R'] / data['TOTVOT16']
req_data['CD'] = data['CD']
req_data.head()

,d_votes,r_votes,CD
0,0.425584,0.508676,1
1,0.337908,0.594626,1
2,0.260996,0.682024,4
3,0.260911,0.684772,4
4,0.354584,0.582430,1


In [16]:
# Creating wasted votes for republican and democratic parties

req_data['d_wasted'] = req_data['d_votes']
req_data['r_wasted'] = req_data['r_votes']
for index in range(len(req_data)):
    if req_data['d_votes'][index] > 0.5:
        req_data['d_wasted'][index] = req_data['d_votes'][index] - 0.5
        req_data['r_wasted'][index] = req_data['r_votes'][index]
        
    elif req_data['r_votes'][index] > 0.5:
        req_data['r_wasted'][index] = req_data['r_votes'][index] - 0.5
        req_data['d_wasted'][index] = req_data['d_votes'][index]
        
req_data.head()

,d_votes,r_votes,CD,d_wasted,r_wasted
0,0.425584,0.508676,1,0.425584,0.008676
1,0.337908,0.594626,1,0.337908,0.094626
2,0.260996,0.682024,4,0.260996,0.182024
3,0.260911,0.684772,4,0.260911,0.184772
4,0.354584,0.582430,1,0.354584,0.082430


In [17]:
fairness_metric(req_data)

0.3149162131754735

In [18]:
action_space_size = 4
state_space_size = 99

q_table = np.zeros((state_space_size, action_space_size))

In [19]:
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0.

## Source: Deeplizard course (https://deeplizard.com/learn/video/jkdXDinWfo8)

Could not get the Reinforcement Learning working since I could not build the custom environment

In [20]:
num_episodes = 100
max_steps_per_episode = 99

learning_rate = 0.1
discount_rate = 0.99

exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.001

In [21]:
rewards_all_episodes = []

In [29]:
class Env():
    def __init__(self, req_data):
        self.action_space = [1, 2, 3, 4]   # For 4 Congressional Districts
        self.states = req_data             # For 99 VTDs
    
    def reset(self):
        self.state = 0
        
    def step(self, state, action):
        return state+1, fairness_metric(self.states[state])

In [30]:
env = Env(req_data)

In [31]:
for episode in range(num_episodes):
    state = 0
    rewards_current_episode = 0

    for step in range(max_steps_per_episode): 
        
        exploration_rate_threshold = random.uniform(0, 1)
        
        if exploration_rate_threshold > exploration_rate:
            action = np.argmax(q_table[state,:])                   # Exploitation
        else:
            action = random.choice([0,1,2,3])                     # Exploration
            
        print(action)

        new_state, reward = env.step(state, action)
        
        # Update Q-table for Q(s,a) using Bellman Optimality Equation
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
        learning_rate * (reward + discount_rate * np.max(q_table[new_state, :]))
        
        state = new_state
        rewards_current_episode += reward 
    
    # Calculating the total fairness 
    
    # Exploration rate decay after each episode where possibility of exploration increases.
    exploration_rate = min_exploration_rate + \
    (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate*episode)
    
    rewards_all_episodes.append(rewards_current_episode)

2


KeyError: 0